# ADA - Project
## Spark

### Test

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext("local", "ADA")

In [ ]:
data = sc.parallelize(range(100))
data.sum()

If you received 4950 as a result, your spark is working well :) Good job !

## Importing the data

In [ ]:
rdd_problem_events = sc.textFile("../data/progfun-002_Problem_Events_with_Info.csv")
for i in rdd_problem_events.take(5): print(i + "\n")

In [ ]:
# Use the filter method to remove the first line
rdd_problem_events = rdd_problem_events.filter(lambda x: not x.startswith('EventID'))
for i in rdd_problem_events.take(5): print(i + "\n")

In [ ]:
# Use the map method to have more workable data
def my_parser(x):
    data = x.split(',')
    return {
        'EventID':data[0],
        'Grade':data[5],
        'TimeStamp':data[6],
        'ProblemID':data[8],
        'ProblemType':data[10],
        'EventType':data[14],
        'SessionUserID':data[16]
    }

rdd_problem_events = rdd_problem_events.map(my_parser)

for i in rdd_problem_events.take(5): 
    print(i)
    print('\n')